In [22]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain



In [21]:
from langchain.callbacks import get_openai_callback


In [49]:
json_payload = {
  "tool": "my-new-tool1",
  "model": "gpt-4o-mini",
  "messages": [
    {"role": "user", "content": "Explain overfitting in machine learning."}
  ],
  "temperature": 0.2,
  "stream": False,
  "websearch": False
}

import json
with open("payload.json", "w") as f:
    json.dump(json_payload, f)


In [53]:
import os
import json
import pandas as pd
import traceback
import requests
from dotenv import load_dotenv
import subprocess

# Load environment variables from .env (if needed)
load_dotenv()

# Input file
file_path = r"C:\Users\a882764\mcqgen\data.txt"
with open(file_path, 'r') as file:
    TEXT = file.read()

# Response JSON template (used in the prompt only)
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
}

# Quiz Generation Prompt
quiz_generation_prompt = f"""
Text: {TEXT}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of 5 multiple choice questions for biology students in simple tone. 
Make sure the questions are not repeated and check all the questions to be conforming to the text.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.
Ensure to make exactly 5 MCQs.

### RESPONSE_JSON
{json.dumps(RESPONSE_JSON)}
"""

# Prepare payload
json_payload = {
    "tool": "my-new-tool1",
    "model": "gpt-4o-mini",
    "messages": [{"role": "user", "content": quiz_generation_prompt}],
    "temperature": 0.2,
    "stream": False,
    "websearch": False
}
with open("payload.json", "w", encoding="utf-8") as f:
    json.dump(json_payload, f, indent=2)


# Call your API
try:
    curl_command = [
        "curl", "https://api.eliza-staging.myatos.net/api/assignmentfromsam/chat",
        "-H", "Content-Type: application/json",
        "-H", "api-key: sK1leK6JUj%ZRY!NMwMi",
        "-H", "accept: */*",
        "-d", "@payload.json"
    ]

    result = subprocess.run(curl_command, capture_output=True, text=True)

    if result.returncode == 0:
        response_text = result.stdout.strip()
        print("✅ Raw Quiz Output:\n", response_text)

        try:
            quiz = json.loads(response_text)

            # Convert to DataFrame
            quiz_table_data = []
            for key, value in quiz.items():
                mcq = value.get("mcq", "")
                options = " | ".join(
                    f"{opt}: {opt_text}" for opt, opt_text in value.get("options", {}).items()
                )
                correct = value.get("correct", "")
                quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

            quiz_df = pd.DataFrame(quiz_table_data)
            display(quiz_df)

        except json.JSONDecodeError:
            print("⚠️ The response is not valid JSON.")
            print(response_text)
    else:
        print("❌ curl command failed:")
        print(result.stderr)

except Exception as e:
    print("❌ An error occurred while calling the API:")
    traceback.print_exc()

✅ Raw Quiz Output:
 {
  "answer": "{\"1\": {\"mcq\": \"What is the scientific study of life called?\", \"options\": {\"a\": \"Physics\", \"b\": \"Chemistry\", \"c\": \"Biology\", \"d\": \"Geology\"}, \"correct\": \"Biology\"}, \"2\": {\"mcq\": \"What is one major theme in biology that explains the unity and diversity of life?\", \"options\": {\"a\": \"Photosynthesis\", \"b\": \"Evolution\", \"c\": \"Cell division\", \"d\": \"Metabolism\"}, \"correct\": \"Evolution\"}, \"3\": {\"mcq\": \"Which organisms are made up of cells that process hereditary information?\", \"options\": {\"a\": \"All organisms\", \"b\": \"Only animals\", \"c\": \"Only plants\", \"d\": \"Only fungi\"}, \"correct\": \"All organisms\"}, \"4\": {\"mcq\": \"Which of the following is a subdiscipline within biology?\", \"options\": {\"a\": \"Molecular biology\", \"b\": \"Astrology\", \"c\": \"Meteorology\", \"d\": \"Geophysics\"}, \"correct\": \"Molecular biology\"}, \"5\": {\"mcq\": \"How long ago did life on Earth emer

Traceback (most recent call last):
  File "C:\Users\a882764\AppData\Local\Temp\ipykernel_18344\3828529284.py", line 99, in <module>
    mcq = value.get("mcq", "")
          ^^^^^^^^^
AttributeError: 'str' object has no attribute 'get'


In [51]:
json_payload = {
  "tool": "my-new-tool1",
  "model": "gpt-4o-mini",
  "messages": [
    {"role": "user", "content": "Explain overfitting in machine learning."}
  ],
  "temperature": 0.2,
  "stream": False,
  "websearch": False
}

import json
with open("payload.json", "w") as f:
    json.dump(json_payload, f)


In [52]:
!curl https://api.eliza-staging.myatos.net/api/assignmentfromsam/chat \
  -H "Content-Type: application/json" \
  -H "api-key: sK1leK6JUj%ZRY!NMwMi" \
  -H "accept: */*" \
  -d @payload.json

{
  "answer": "Overfitting in machine learning refers to a model that has learned the training data too well, capturing noise and outliers as if they were significant patterns. This results in a model that performs well on training data but poorly on unseen or test data. Here are key points about overfitting:\n\n- **Definition**: Overfitting occurs when a model becomes too complex, with too many parameters relative to the amount of data, allowing it to model random noise in the training data rather than the intended outputs.\n\n- **Indicators**: The primary indicator of overfitting is a significant discrepancy between the model's performance on training data versus its performance on validation or test data. \n\n- **Causes**:\n  - **Complex models**: Using highly complex models with many features or layers can lead to overfitting.\n  - **Insufficient data**: When there is not enough training data, the model might capture noise rather than generalizable patterns.\n  - **Data quality**: 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100   192    0     0  100   192      0     94  0:00:02  0:00:02 --:--:--    95
100   192    0     0  100   192      0     63  0:00:03  0:00:03 --:--:--    63
100   192    0     0  100   192      0     47  0:00:04  0:00:04 --:--:--    47
100   192    0     0  100   192      0     37  0:00:05  0:00:05 --:--:--    38
100   192    0     0  100   192      0     31  0:00:06  0:00:06 --:--:--    38
100   192    0     0  100   192      0     27  0:00:07  0:00:07 --:--:--     0
100  4829  100  4637  100   192    632     26  0:00:07  0:00:07 --:--:--  1081
